In [98]:
import requests
import re
from bs4 import BeautifulSoup

url = "https://paulgraham.com/{}"


def scrape_essay_urls(url):
    page = requests.get(url.format("articles.html"))
    soup = BeautifulSoup(page.content, "html.parser")
    links = soup.findAll("a")

    essay_urls = {
        url.format(link["href"])
        for link in links
        if (link["href"].endswith(".html") or "https://" in link["href"])
        and link["href"] not in {"index.html", "rss.html"}
    }
    return essay_urls


def scrape_essays_texts(urls):
    essay_data = []
    for essay_url in list(urls)[0:3]:
        page = requests.get(essay_url)
        soup = BeautifulSoup(page.content, "html.parser")
        raw_text = soup.get_text(separator="\n", strip=True)
        raw_text_without_notes = raw_text.split("Notes")[0]
        clean_text = re.sub(r"\[\n\d+\n\]", "", raw_text_without_notes)
        data = {
            "title": clean_text.split("\n")[0],
            "date": clean_text.split("\n")[1],
            "essay_text": "\n".join(clean_text.split("\n")[2:]),
        }
        essay_data.append(data)
    return essay_data

In [99]:
essay_urls = scrape_essay_urls(url)
essay_texts = scrape_essays_texts(essay_urls)